In [1]:
from selenium import webdriver as wb
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time # 딜레이 기능 - sleep(초)

# 파일 다루는 라이브러리
#파일or폴더 생성, 삭제, 수정, 파일or폴더 존재 여부 확인
import os

# 판다스
import pandas as pd

In [2]:
GN0100_df = pd.read_excel('./GN0100_50차트.xlsx') 
GN0100_df

,곡명,가수명
0,네가 보고 싶은 건 자연스러운 거겠지,신예영
1,악몽 (Art. 새빛)(With. MINE),Anonymous Artists(어나니머스 아티스트)
2,가로등,서로 (Suh Ro)
3,그게 뭐 어때서요,유하정
4,그대가 내 안에 박혔다(그내박),순순희(기태)
5,위로가 된다면,이아람
6,다만 널 사랑해서 그래,윤원
7,사랑이 내 삶에 들어왔다,명지
8,찌질의 역사,김우성
9,알고 싶어요,A+B


In [3]:
GN0100_df.iloc[0][0]

'네가 보고 싶은 건 자연스러운 거겠지'

In [4]:
# 가수명-곡명으로 검색한 url 리스트만들기
url_lst = []
for i in range(len(GN0100_df)):
    song = GN0100_df.iloc[i][0] # 곡명 담기
    singer = GN0100_df.iloc[i][1] # 가수명 담기
    url = f'https://www.youtube.com/results?search_query={singer}+-+{song}' # 가수명-곡명으로 검색한 url
    url_lst.append(url) # url리스트에 담아주기

In [5]:
# 영상url 리스트 만들기
href_lst = []

for i in range(len(url_lst)):
    driver = wb.Chrome()
    driver.get(url_lst[i])
    time.sleep(2)
    
    # 검색하여 만들어지 url에서 첫 번째 영상 접근
    aTag = driver.find_elements(By.CSS_SELECTOR, value = 'ytd-video-renderer a#thumbnail')
    # 첫 번째 영상의 링크 href변수에 담아주기
    href = aTag[0].get_attribute('href')
    # href_lst 리스트 생성
    href_lst.append(href)

In [6]:
href_lst

['https://www.youtube.com/watch?v=bZ4Qc5Qbfcg',
 'https://www.youtube.com/watch?v=zEjbESib46o',
 'https://www.youtube.com/watch?v=W7DpFq0pUHE',
 'https://www.youtube.com/watch?v=O3mKFrcXJ0w',
 'https://www.youtube.com/watch?v=RyiFowxqN-Q',
 'https://www.youtube.com/watch?v=5eIIDzrwbQ0',
 'https://www.youtube.com/watch?v=e7AyG-lc11M',
 'https://www.youtube.com/watch?v=vzuQC9Va1Dg',
 'https://www.youtube.com/watch?v=GnxmGqvH03g',
 'https://www.youtube.com/watch?v=fEXWyjcVRWA',
 'https://www.youtube.com/watch?v=4yYCvNjUhS8&pp=ygUl6rmA66qp7ZWYIC0g7IKs656RLCAoZmVhdC4g67Cx7ISc7ZiEKQ%3D%3D',
 'https://www.youtube.com/watch?v=diO0nqGHJw8',
 'https://www.youtube.com/shorts/VvkMo_GFGjg',
 'https://www.youtube.com/watch?v=LxhUwLVSOqg',
 'https://www.youtube.com/watch?v=xTJ-HyXqrFo',
 'https://www.youtube.com/watch?v=-gxjbz_OU0I',
 'https://www.youtube.com/watch?v=zA0CvFgJDb4',
 'https://www.youtube.com/watch?v=7AHjnBiviV8',
 'https://www.youtube.com/watch?v=93nAfKzBITs',
 'https://www.youtube.com

In [7]:
GN0100_df['링크'] = href_lst
GN0100_df

,곡명,가수명,링크
0,네가 보고 싶은 건 자연스러운 거겠지,신예영,https://www.youtube.com/watch?v=bZ4Qc5Qbfcg
1,악몽 (Art. 새빛)(With. MINE),Anonymous Artists(어나니머스 아티스트),https://www.youtube.com/watch?v=zEjbESib46o
2,가로등,서로 (Suh Ro),https://www.youtube.com/watch?v=W7DpFq0pUHE
3,그게 뭐 어때서요,유하정,https://www.youtube.com/watch?v=O3mKFrcXJ0w
4,그대가 내 안에 박혔다(그내박),순순희(기태),https://www.youtube.com/watch?v=RyiFowxqN-Q
5,위로가 된다면,이아람,https://www.youtube.com/watch?v=5eIIDzrwbQ0
6,다만 널 사랑해서 그래,윤원,https://www.youtube.com/watch?v=e7AyG-lc11M
7,사랑이 내 삶에 들어왔다,명지,https://www.youtube.com/watch?v=vzuQC9Va1Dg
8,찌질의 역사,김우성,https://www.youtube.com/watch?v=GnxmGqvH03g
9,알고 싶어요,A+B,https://www.youtube.com/watch?v=fEXWyjcVRWA
